# Load Library

In [1]:
# imports for neural network
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset

# imports for vision tasks
import torchvision
import torchvision.models as models
import torchvision.transforms.v2 as transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import pydicom

# imports for preparing dataset
import os
import shutil
import zipfile
import pandas as pd
from skimage import io
from PIL import Image
import numpy as np
import timm

# imports for visualizations
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

from tqdm import tqdm
from pathlib import Path
import json
from typing import Dict, Tuple, List, Optional, Union, Any
import platform
from enum import Enum
import timm
import time

from sklearn.metrics import (
    roc_auc_score, 
    precision_recall_curve, 
    auc,
    accuracy_score,
    confusion_matrix
)

# Load Data

## Define Data Structure Class

In [2]:
class SpinalConditionDataset(Dataset):
    """Custom Dataset class with optimized device handling"""
    def __init__(
        self, 
        images: torch.Tensor, 
        labels: torch.Tensor, 
        metadata: pd.DataFrame,
        device: Optional[torch.device] = None
    ):
        if device is None:
            self.device, self.device_type = get_best_available_device()
        else:
            self.device = device
            self.device_type = device.type
        
        # Store as CPU tensors initially
        self.images = images
        self.labels = labels
        self.metadata = metadata
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        # Move individual items to device when accessed
        image = self.images[idx].to(self.device)
        label = self.labels[idx].to(self.device)
        return image, label

## Define Premade Function

### Getting Best Devices

In [3]:
def get_best_available_device() -> Tuple[torch.device, str]:
    """
    Detect and return the best available device for tensor operations.
    
    Returns:
        Tuple[torch.device, str]: Device object and device type string
    """
    device_type = "cpu"
    
    # Check for CUDA
    if torch.cuda.is_available():
        device_type = "cuda"
        
    # Check for Apple M1/M2 MPS (Metal Performance Shaders)
    elif platform.processor().startswith('arm') and platform.system() == 'Darwin' and \
         hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        device_type = "mps"
        
    # Check for TPU (if torch_xla is available)
    try:
        import torch_xla.core.xla_model as xm
        device_type = "tpu"
    except ImportError:
        pass
        
    # Create device object based on type
    if device_type == "tpu":
        try:
            device = xm.xla_device()
        except NameError:
            device = torch.device("cpu")
            device_type = "cpu"
    else:
        device = torch.device(device_type)
        
    return device, device_type

### Data Loading Function

In [4]:
def load_processed_dataset(base_load_path, verbose=False):
    """
    Load the processed dataset from disk.
    
    Args:
        base_load_path (str): Path where the dataset is saved
    
    Returns:
        dict: Loaded condition data
        dict: Configuration data
    """
    base_path = Path(base_load_path)
    
    if not base_path.exists():
        raise ValueError(f"Dataset path {base_path} does not exist")
    
    # Load configuration
    with open(base_path / 'config.json', 'r') as f:
        config = json.load(f)
    
    # Initialize condition data dictionary
    loaded_condition_data = {}
    
    # Load data for each condition
    for condition in config['core_conditions']:
        condition_path = base_path / condition
        
        if condition_path.exists():
            loaded_condition_data[condition] = {
                'images': torch.load(condition_path / 'images.pt', weights_only=False),
                'labels': torch.load(condition_path / 'labels.pt', weights_only=False),
                'metadata': pd.read_pickle(condition_path / 'metadata.pkl')
            }
    
    if verbose is True :
        print("\nDataset successfully loaded")
        print("\nDataset Summary:")
        for condition, data in loaded_condition_data.items():
            print(f"\n{condition}:")
            print(f"Total samples: {len(data['images'])}")
            label_dist = torch.bincount(data['labels'])
            for severity, idx in config['severity_mapping'].items():
                if idx < len(label_dist):
                    print(f"  {severity}: {label_dist[idx].item()}")
    
    return loaded_condition_data, config

### Data Splitting Function

In [5]:
def create_train_val_split(
    condition_data: Dict,
    device: Optional[torch.device] = None,
    val_ratio: float = 0.2,
    seed: int = 42
) -> Dict[str, Dict[str, Dict]]:
    """
    Split the dataset into training and validation sets with automatic device support.
    
    Args:
        condition_data (Dict): Dictionary containing data for each condition
        device (Optional[torch.device]): Device to store the tensors on
        val_ratio (float): Ratio of validation set size to total dataset size
        seed (int): Random seed for reproducibility
    
    Returns:
        Dict: Dictionary containing train and val splits for each condition
    """
    torch.manual_seed(seed)
    
    split_data = {
        'train': {},
        'val': {}
    }
    
    for condition, data in condition_data.items():
        dataset_size = len(data['images'])
        val_size = int(dataset_size * val_ratio)
        train_size = dataset_size - val_size
        
        # Create full dataset with device specification
        full_dataset = SpinalConditionDataset(
            data['images'],
            data['labels'],
            data['metadata'],
            device=device
        )
        
        # Split dataset
        train_dataset, val_dataset = random_split(
            full_dataset, 
            [train_size, val_size],
            generator=torch.Generator().manual_seed(seed)
        )
        
        split_data['train'][condition] = train_dataset
        split_data['val'][condition] = val_dataset
        
    return split_data

### Data Loader Creator

In [6]:
def create_dataloaders(
    split_data: Dict[str, Dict[str, Dataset]],
    device_type: str,
    batch_size: int = 32,
    num_workers: int = 4,
    shuffle_train: bool = True
) -> Dict[str, Dict[str, DataLoader]]:
    """
    Create DataLoader objects optimized for the detected device.
    
    Args:
        split_data (Dict): Dictionary containing train and val splits
        device_type (str): Type of device being used
        batch_size (int): Batch size for DataLoader
        num_workers (int): Number of worker processes
        shuffle_train (bool): Whether to shuffle training data
    
    Returns:
        Dict: Dictionary containing DataLoader objects for each split and condition
    """
    dataloaders = {
        'train': {},
        'val': {}
    }
    
    # Optimize DataLoader settings based on device
    if device_type == "cuda":
        pin_memory = False  # Data is already on GPU
    elif device_type == "tpu":
        pin_memory = False
        num_workers = 0  # TPU often works better with synchronous loading
    else:  # CPU or MPS
        pin_memory = True
    
    for split in ['train', 'val']:
        for condition, dataset in split_data[split].items():
            dataloaders[split][condition] = DataLoader(
                dataset,
                batch_size=batch_size,
                shuffle=(shuffle_train and split == 'train'),
                num_workers=num_workers,
                pin_memory=pin_memory
            )
    
    return dataloaders

### Getting Batch Size

In [7]:
def get_device_specific_batch_size(device_type: str, base_batch_size: int) -> int:
    """
    Adjust batch size based on device type and available memory.
    
    Args:
        device_type (str): Type of device being used
        base_batch_size (int): Requested batch size
        
    Returns:
        int: Adjusted batch size
    """
    if device_type == "cuda":
        # Get available GPU memory and adjust batch size if needed
        gpu_memory = torch.cuda.get_device_properties(0).total_memory
        if gpu_memory < 8 * (1024**3):  # Less than 8GB
            return min(base_batch_size, 16)
    elif device_type == "tpu":
        # TPUs often work better with larger batch sizes
        return max(base_batch_size, 128)
    
    return base_batch_size

### Usage Function

In [8]:
def load_and_prepare_data(
    base_load_path: str,
    batch_size: int = 32,
    val_ratio: float = 0.2,
    num_workers: int = 4,
    seed: int = 42,
    verbose: bool = False
) -> Tuple[Dict, Dict, Dict, torch.device]:
    """
    Load and prepare data loaders with automatic device detection and optimization.
    
    Args:
        base_load_path (str): Path to dataset
        batch_size (int): Base batch size for DataLoader
        val_ratio (float): Validation set ratio
        num_workers (int): Number of worker processes
        seed (int): Random seed
        verbose (bool): Whether to print dataset information
    
    Returns:
        Tuple containing:
        - Dictionary of dataloaders
        - Dictionary of split datasets
        - Configuration dictionary
        - Device being used
    """
    # Detect best available device
    device, device_type = get_best_available_device()
    
    if verbose:
        print(f"\nUsing device: {device} ({device_type})")
    
    # Adjust batch size for device
    adjusted_batch_size = get_device_specific_batch_size(device_type, batch_size)
    if verbose and adjusted_batch_size != batch_size:
        print(f"Adjusted batch size from {batch_size} to {adjusted_batch_size} for {device_type}")
    
    # Load the dataset
    condition_data, config = load_processed_dataset(base_load_path, verbose=verbose)
    
    # Create train/val split
    split_data = create_train_val_split(
        condition_data,
        device=device,
        val_ratio=val_ratio,
        seed=seed
    )
    
    # Create dataloaders
    dataloaders = create_dataloaders(
        split_data,
        device_type=device_type,
        batch_size=adjusted_batch_size,
        num_workers=num_workers
    )
    
    if verbose:
        print("\nDataset Summary:")
        for split in ['train', 'val']:
            print(f"\n{split.capitalize()} set sizes:")
            for condition in config['core_conditions']:
                if condition in split_data[split]:
                    print(f"{condition}: {len(split_data[split][condition])}")
        
        # Print memory usage information
        if device_type == "cuda":
            print("\nGPU Memory Usage:")
            print(f"Allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
            print(f"Cached: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")
        elif device_type == "tpu":
            try:
                import torch_xla.debug.metrics as met
                print("\nTPU Memory Usage:")
                print(met.metrics_report())
            except ImportError:
                pass
    
    return dataloaders, split_data, config, device

## Load & Create Dataloader

In [9]:
dataloaders, split_data, config, device = load_and_prepare_data(
    base_load_path="/kaggle/input/degenerative-spine-image-classificaton/processed_spine_dataset",
    batch_size=32,
    val_ratio=0.2,
    verbose=True,
    num_workers=0
)


Using device: cpu (cpu)

Dataset successfully loaded

Dataset Summary:

spinal_canal_stenosis:
Total samples: 9753
  Normal/Mild: 8552
  Moderate: 732
  Severe: 469

left_neural_foraminal_narrowing:
Total samples: 9860
  Normal/Mild: 7671
  Moderate: 1792
  Severe: 397

right_neural_foraminal_narrowing:
Total samples: 9829
  Normal/Mild: 7684
  Moderate: 1767
  Severe: 378

left_subarticular_stenosis:
Total samples: 9603
  Normal/Mild: 6857
  Moderate: 1834
  Severe: 912

right_subarticular_stenosis:
Total samples: 9612
  Normal/Mild: 6862
  Moderate: 1825
  Severe: 925

Dataset Summary:

Train set sizes:
spinal_canal_stenosis: 7803
left_neural_foraminal_narrowing: 7888
right_neural_foraminal_narrowing: 7864
left_subarticular_stenosis: 7683
right_subarticular_stenosis: 7690

Val set sizes:
spinal_canal_stenosis: 1950
left_neural_foraminal_narrowing: 1972
right_neural_foraminal_narrowing: 1965
left_subarticular_stenosis: 1920
right_subarticular_stenosis: 1922


In [10]:
from torch.utils.data import Subset

def get_validation_subset(val_data: Dataset, subset_size: int, seed: int = 42) -> Subset:
    """
    Extract a subset from the validation dataset.

    Args:
        val_data (Dataset): The validation dataset.
        subset_size (int): The number of samples to include in the subset.
        seed (int): Random seed for reproducibility.

    Returns:
        Subset: A subset of the validation dataset.
    """
    torch.manual_seed(seed)
    indices = torch.randperm(len(val_data))[:subset_size]  # Randomly sample indices
    return Subset(val_data, indices)


In [11]:
# Define conditions
conditions = [
    "spinal_canal_stenosis",
    "left_neural_foraminal_narrowing",
    "right_neural_foraminal_narrowing",
    "left_subarticular_stenosis",
    "right_subarticular_stenosis"
]

subset_size = 100
val_subset_loaders = {}

for condition in conditions:
    val_dataset = split_data['val'][condition]
    val_subset = get_validation_subset(val_dataset, subset_size)
    val_subset_loaders[condition] = DataLoader(
        val_subset,
        batch_size=32,
        shuffle=False,
        num_workers=0,
        pin_memory=False
    )

# Spinal Model

In [12]:
import torch
import torch.nn as nn
import timm
from enum import Enum
from typing import Any, Dict, Tuple
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau


class ModelArchitecture(str, Enum):
    """Supported model architectures"""
    INCEPTION_V4 = "inception_v4"
    EFFICIENTNET = "efficientnet_b0"
    EFFICIENTNET_V2 = "efficientnetv2_s"
    VGG16 = "vgg16"


class SpinalModel(nn.Module):
    """Enhanced neural network model for spinal condition classification"""
    def __init__(
        self,
        architecture: ModelArchitecture,
        num_classes: int,
        pretrained: bool = True,
        dropout_rate: float = 0.5,
        weight_decay: float = 5e-3,
        unfreeze_layers: int = 0,
    ):
        super().__init__()
        
        self.architecture = architecture
        self.num_classes = num_classes
        self.weight_decay = weight_decay
        
        # Get the best available device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Create the backbone model
        self.backbone = self._create_backbone(pretrained)
        
        # Apply partial freezing of layers
        self.partial_freeze_backbone(unfreeze_layers)
        
        # Get the number of features from the backbone
        num_features = self._get_num_features()
        
        # Enhanced classifier with residual connections
        self.classifier = self._create_classifier(num_features, dropout_rate)
        
        # Initialize weights
        self._initialize_weights()
        
        # Move model to device
        self.to(self.device)

    def _create_backbone(self, pretrained: bool) -> nn.Module:
        """Create the backbone model"""
        if self.architecture == ModelArchitecture.INCEPTION_V4:
            model = timm.create_model('inception_v4.tf_in1k', pretrained=pretrained, num_classes=0)
        elif self.architecture == ModelArchitecture.EFFICIENTNET:
            model = timm.create_model('efficientnet_b0.ra4_e3600_r224_in1k', pretrained=pretrained, num_classes=0)
        elif self.architecture == ModelArchitecture.EFFICIENTNET_V2:
            model = timm.create_model('tf_efficientnetv2_s.in21k_ft_in1k', pretrained=pretrained, num_classes=0)
        elif self.architecture == ModelArchitecture.VGG16:
            model = timm.create_model('vgg16.tv_in1k', pretrained=pretrained, num_classes=0)
        else:
            raise ValueError(f"Unsupported architecture: {self.architecture}")
        return model

    def _get_num_features(self) -> int:
        """Get number of features from backbone"""
        if self.architecture == ModelArchitecture.INCEPTION_V4:
            return 1536
        elif self.architecture in [ModelArchitecture.EFFICIENTNET, ModelArchitecture.EFFICIENTNET_V2]:
            return 1280
        elif self.architecture == ModelArchitecture.VGG16:
            return 4096
        raise ValueError(f"Unsupported architecture: {self.architecture}")

    def _create_classifier(self, num_features: int, dropout_rate: float) -> nn.Sequential:
        """Create classifier with residual connections"""
        return nn.Sequential(
            nn.Identity(),
            nn.Flatten(),
            nn.BatchNorm1d(num_features),
            nn.Dropout(p=dropout_rate),
            
            # First dense block
            self._create_dense_block(num_features, 512, dropout_rate),
            
            # Second dense block
            self._create_dense_block(512, 256, dropout_rate),
            
            # Final classification
            nn.BatchNorm1d(256),
            nn.Dropout(p=dropout_rate),
            nn.Linear(256, self.num_classes)
        )

    def _create_dense_block(self, in_features: int, out_features: int, dropout_rate: float) -> nn.Sequential:
        """Create a dense block with batch norm and dropout"""
        return nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.ReLU(),
            nn.BatchNorm1d(out_features),
            nn.Dropout(p=dropout_rate)
        )

    def _initialize_weights(self):
        """Initialize model weights"""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def partial_freeze_backbone(self, unfreeze_layers: int):
        """Partially freeze backbone layers"""
        if unfreeze_layers == 0:
            for param in self.backbone.parameters():
                param.requires_grad = False
            return
            
        parameters = list(self.backbone.named_parameters())
        total_layers = len(parameters)
        
        # Freeze all layers first
        for param in self.backbone.parameters():
            param.requires_grad = False
            
        # Unfreeze the specified number of layers from the bottom
        for i in range(max(0, total_layers - unfreeze_layers), total_layers):
            parameters[i][1].requires_grad = True

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass"""
        x = x.to(self.device)
        features = self.backbone(x)
        return self.classifier(features)

def create_model(
    architecture: str,
    num_classes: int,
    pretrained: bool = True,
    dropout_rate: float = 0.5,
    weight_decay: float = 5e-3,
    unfreeze_layers: int = 0,
    verbose: bool = False
) -> Tuple[SpinalModel, Dict[str, Any]]:
    """Create a model instance"""
    try:
        arch = ModelArchitecture(architecture.lower())
    except ValueError:
        raise ValueError(
            f"Unsupported architecture: {architecture}. "
            f"Supported architectures: {[a.value for a in ModelArchitecture]}"
        )
    
    # Create model
    model = SpinalModel(
        architecture=arch,
        num_classes=num_classes,
        pretrained=pretrained,
        dropout_rate=dropout_rate,
        weight_decay=weight_decay,
        unfreeze_layers=unfreeze_layers
    )
    
    # Get preprocessing parameters
    preprocess_params = {
        'image_size': 299 if arch == ModelArchitecture.INCEPTION_V4 else 224,
        'mean': [0.485, 0.456, 0.406],
        'std': [0.229, 0.224, 0.225],
        'device': model.device
    }

    if verbose:
        print(f"\nModel Configuration:")
        print(f"Architecture: {arch.value}")
        # print(f"Device: {preprocess_params['device']} ({preprocess_params['device_type']})")
        print(f"Input size: {preprocess_params['image_size']}x{preprocess_params['image_size']}")
        print(f"Number of classes: {num_classes}")
        print(f"Pretrained: {pretrained}")
        
        # Print model summary if torchinfo is available
        try:
            from torchsummary import summary
            # Assuming 'model' is your SpinalModel
            input_size = (3, preprocess_params['image_size'], preprocess_params['image_size'])
            summary(model, input_size=input_size)
        except ImportError:
            if verbose:
                print("\nInstall torchsummary for detailed model summary")
    
    return model, preprocess_params

In [13]:
model, preprocess_params = create_model(
    architecture="vgg16",  # or "efficientnet" or "efficientnet_v2"
    num_classes=3,
    pretrained=True,
    verbose=True,
    unfreeze_layers=15
)

model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]


Model Configuration:
Architecture: vgg16
Input size: 224x224
Number of classes: 3
Pretrained: True
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
   

In [14]:
checkpoint = torch.load('/kaggle/input/rsna-lumbar-dataset-vgg16-class/checkpoints/checkpoint_best.pth', map_location=model.device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()  # If using for inference

/tmp/ipykernel_26/3626930169.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('/kaggle/input/rsna-lumbar-dataset-vgg16-class/checkpoints/checkpoin

SpinalModel(
  (backbone): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
     

# Test

In [15]:
def test_model(
   model: nn.Module,
   test_loaders: Dict[str, DataLoader],
   num_classes: int,
   save_dir: str,
   threshold: float = 0.5,
   device: Optional[torch.device] = None
) -> Dict:
   model.eval()
   device = device or model.device
   metrics = {}
   
   Path(save_dir).mkdir(parents=True, exist_ok=True)
   
   for condition, loader in test_loaders.items():
       all_preds, all_labels, all_probs, all_images = [], [], [], []
       
       with torch.no_grad():
           for inputs, labels in loader:
               inputs = inputs.to(device, non_blocking=True)
               labels = labels.to(device, non_blocking=True)
               
               outputs = model(inputs)
               probs = torch.softmax(outputs, dim=1)
               preds = torch.argmax(probs, dim=1)
               
               all_preds.extend(preds.cpu().numpy())
               all_probs.extend(probs.cpu().numpy())
               all_labels.extend(labels.cpu().numpy())
               all_images.extend(inputs.cpu().numpy())
       
       all_preds = np.array(all_preds)
       all_probs = np.array(all_probs)
       all_labels = np.array(all_labels)
       all_images = np.array(all_images)
       
       # Visualization
       num_samples = min(10, len(all_images))
       fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5*num_samples))
       
       for i in range(num_samples):
           img = all_images[i].transpose(1, 2, 0)
           img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
           img = np.clip(img, 0, 1)
           
           axes[i, 0].imshow(img)
           axes[i, 0].set_title(f'Image {i}')
           
           axes[i, 1].bar(range(num_classes), np.eye(num_classes)[all_labels[i]])
           axes[i, 1].set_title(f'True: {all_labels[i]}')
           
           axes[i, 2].bar(range(num_classes), all_probs[i])
           axes[i, 2].set_title(f'Pred: {all_preds[i]} ({all_probs[i][all_preds[i]]:.2f})')
           
       plt.tight_layout()
       plt.savefig(os.path.join(save_dir, f'{condition}_results.png'))
       plt.close()
       
       # Metrics
       metrics[condition] = {
           'accuracy': accuracy_score(all_labels, all_preds),
           'confusion_matrix': confusion_matrix(all_labels, all_preds),
           'per_class_accuracy': [],
           'roc_auc_scores': [],
           'pr_auc_scores': []
       }
       
       for i in range(num_classes):
           class_labels = (all_labels == i).astype(int)
           class_probs = all_probs[:, i]
           class_preds = (all_preds == i).astype(int)
           
           metrics[condition]['roc_auc_scores'].append(roc_auc_score(class_labels, class_probs))
           precision, recall, _ = precision_recall_curve(class_labels, class_probs)
           metrics[condition]['pr_auc_scores'].append(auc(recall, precision))
           metrics[condition]['per_class_accuracy'].append(accuracy_score(class_labels, class_preds))
       
       metrics[condition].update({
           'avg_roc_auc': np.mean(metrics[condition]['roc_auc_scores']),
           'avg_pr_auc': np.mean(metrics[condition]['pr_auc_scores']),
           'avg_accuracy': np.mean(metrics[condition]['per_class_accuracy'])
       })
       
       print(f"\nResults for {condition}:")
       print(f"Average ROC AUC: {metrics[condition]['avg_roc_auc']:.4f}")
       print(f"Average PR AUC: {metrics[condition]['avg_pr_auc']:.4f}")
       print(f"Average Accuracy: {metrics[condition]['avg_accuracy']:.4f}")
       print("\nConfusion Matrix:")
       print(metrics[condition]['confusion_matrix'])
   
   return metrics

In [16]:
# Usage
test_metrics = test_model(
   model=model,
   test_loaders=val_subset_loaders,
   num_classes=3,
   save_dir='./checkpoints'
)


Results for spinal_canal_stenosis:
Average ROC AUC: 0.7064
Average PR AUC: 0.3953
Average Accuracy: 0.6600

Confusion Matrix:
[[42 31 14]
 [ 1  5  3]
 [ 1  1  2]]

Results for left_neural_foraminal_narrowing:
Average ROC AUC: 0.7022
Average PR AUC: 0.4028
Average Accuracy: 0.5867

Confusion Matrix:
[[32 39 12]
 [ 1  4  8]
 [ 0  2  2]]

Results for right_neural_foraminal_narrowing:
Average ROC AUC: 0.5941
Average PR AUC: 0.4057
Average Accuracy: 0.5400

Confusion Matrix:
[[24 37 16]
 [ 4  5  9]
 [ 1  2  2]]

Results for left_subarticular_stenosis:
Average ROC AUC: 0.8435
Average PR AUC: 0.5966
Average Accuracy: 0.7667

Confusion Matrix:
[[46 15  9]
 [ 0 14  9]
 [ 0  2  5]]

Results for right_subarticular_stenosis:
Average ROC AUC: 0.7691
Average PR AUC: 0.5216
Average Accuracy: 0.6867

Confusion Matrix:
[[37 27  4]
 [ 1 10 11]
 [ 0  4  6]]
